<span style="color:gray">
Fraunhofer Institute for Integrated Circuits IIS, Division Engineering of Adaptive Systems EAS<br>
Münchner Straße 16, 01187 Dresden, Germany
</span>

---

## ESB - Energy Saving by Blockchain

---

## Detection of Electric Vehicles and Photovoltaic Systems in Smart Meter Data

---

# 6: Further Figures

The source code for the remaining figures found in the paper is presented in this notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import operator
import urllib.request
import base64
import json
import seaborn as sns

from matplotlib import pyplot as plt
from dateutil import parser
from sklearn.neighbors import KernelDensity

#### Total energy balance (New York dataset, customer_ID: 27)

In [ ]:
path=''

newyork = pd.read_csv(path + '1hour_data_newyork.csv', header=0, sep=',',decimal=".")

newyork['local_15min']=newyork['local_15min'].apply(lambda x: parser.parse(x))

x = newyork.loc[:71,'local_15min'].tolist()
y = newyork.loc[:71,'grid'].tolist()
y1 = newyork.loc[:71,'car1'].tolist()
y2 = newyork.loc[:71,'solar'].tolist()
y2 = list(map(operator.neg, y2))

fig = plt.figure()
plt.plot(x,y, linewidth=1.0, label = "Energy consumption")
plt.plot(x,y1, linewidth=1.0, label = "Electric vehicle")
plt.plot(x,y2, linewidth=1.0, label = "Solar cell")
plt.gcf().autofmt_xdate()
plt.grid(True)
plt.xlabel("Time")
plt.ylabel("Total energy balance [kWh]")
plt.title("Total energy balance (New York dataset, customer_ID: 27)")
plt.ylim(-10, 10)
plt.legend()
plt.show()

#### ACN Charging Session

In [ ]:
access_token = ''

charging_session = pd.DataFrame()
_id='5c2e8674f9af8b13dab075ff'

credentials = ('%s:%s' % (access_token, ''))
encoded_credentials = base64.b64encode(credentials.encode('ascii'))

req = urllib.request.Request(f'https://ev.caltech.edu/api/v1/sessions/caltech/ts/?where=_id=="{_id}"')
req.add_header('Authorization', 'Basic %s' % encoded_credentials.decode("ascii"))
data = json.loads(urllib.request.urlopen(req).read())

charging_session=charging_session.append({'time':data['_items'][0]['chargingCurrent']['timestamps'],'ts':data['_items'][0]['chargingCurrent']['current']}, ignore_index=True)
charging_session = pd.concat((pd.DataFrame(np.stack(charging_session['time'].values).T, columns=['time']), pd.DataFrame(np.stack(charging_session['ts'].values).T, columns=['ts'])), axis=1) 
    
charging_session['time'] = pd.to_datetime(charging_session['time'], format='%a, %d %b %Y %H:%M:%S %Z') 
charging_session = charging_session.set_index('time').groupby(pd.Grouper(freq='1min')).agg({'ts':'mean'}).reset_index()
charging_session['time']=charging_session['time'].dt.time.astype('str').apply(lambda x: parser.parse(x))

time=charging_session['time'].tolist()
ts=charging_session['ts'].tolist()


fig = plt.figure()
plt.plot(time,ts, linewidth=1, label = "Energy consumption")
plt.gcf().autofmt_xdate()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.grid(True)
plt.xlabel("Time")
plt.ylabel("Charging power [kWh]")
plt.title("Session_id: 5c2e8674f9af8b13dab075ff")
plt.ylim(0, 20)
plt.legend()
plt.show()

#### Comparison of kernel density estimates of the synthetic dataset from California with and without electric vehicle charging.

In [ ]:
path=''

california = pd.read_csv (path + '1hour_data_california.csv', header=0, sep=',',decimal=".")
california_with_EV_id_203 = california.loc[:743,'grid']
california_without_EV_id_203 = california.loc[:743,'grid'] - california.loc[:743,'car1']

fig, ax = plt.subplots()

X=california_with_EV_id_203.to_numpy()[:, np.newaxis]
N=len(X)
X_plot = np.linspace(-13, 35, 1000)[:, np.newaxis]
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(X)
log_dens = kde.score_samples(X_plot)
a=np.exp(log_dens)
ax.plot(X_plot[:, 0], np.exp(log_dens), color='navy', label="with EV")
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), '+k', color='navy')

X=california_without_EV_id_203.to_numpy()[:, np.newaxis]
N=len(X)
X_plot = np.linspace(-5, 35, 1000)[:, np.newaxis]
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(X)
log_dens = kde.score_samples(X_plot)
ax.plot(X_plot[:, 0], np.exp(log_dens), color='darkorange', label="without EV")
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), '+k', color='darkorange')

fig.suptitle('Kernel Density Estimation (California dataset)', fontsize=12)
ax.legend(loc='upper right')
ax.set_xlim(-3, 20)
ax.set_ylim(-0.02, 0.8)
ax.set_xlabel("Charging power [kWh]") 
ax.set_ylabel("Density") 
plt.show()

#### Histrogram of the charging capacities of charging processes of the ACN dataset

In [ ]:
access_token = ''

def concat_charging_cycles(access_token, location, number_of_charge_cycles):

    credentials = ('%s:%s' % (access_token, ''))
    encoded_credentials = base64.b64encode(credentials.encode('ascii'))

    location_concat = pd.DataFrame()
 
    for i in range(1, number_of_charge_cycles):
        req = urllib.request.Request(f'https://ev.caltech.edu/api/v1/sessions/{location}?&page={i}')
        req.add_header('Authorization', 'Basic %s' % encoded_credentials.decode("ascii"))
        data = json.loads(urllib.request.urlopen(req).read())

        location_id=data['_items'][0]['_id']
        print(f"{i}. {location}_id: {location_id}")
    
        req = urllib.request.Request(f'https://ev.caltech.edu/api/v1/sessions/{location}/ts/?where=_id=="{location_id}"')
        req.add_header('Authorization', 'Basic %s' % encoded_credentials.decode("ascii"))
        data = json.loads(urllib.request.urlopen(req).read())
    
        location_ts = pd.DataFrame()
        
        if data['_items'][0]['chargingCurrent'] != None:
        
            location_ts=location_ts.append({'ts':data['_items'][0]['chargingCurrent']['current']}, ignore_index=True)
            location_ts=pd.DataFrame(np.stack(location_ts['ts'].values)).T
 
            location_concat = pd.concat([location_concat,location_ts]).reset_index(drop=True)
        
    return location_concat

In [ ]:
office001_ts_concat = concat_charging_cycles(access_token, 'office001', 60)  
office001_ts_concat = office001_ts_concat[office001_ts_concat != 0]
office001_ts_concat = office001_ts_concat.dropna().reset_index(drop=True)

jpl_ts_concat = concat_charging_cycles(access_token, 'jpl', 150)  
jpl_ts_concat = jpl_ts_concat[jpl_ts_concat != 0]
jpl_ts_concat = jpl_ts_concat.dropna().reset_index(drop=True)

caltech_ts_concat = concat_charging_cycles(access_token, 'caltech', 60)  
caltech_ts_concat = caltech_ts_concat[caltech_ts_concat != 0]
caltech_ts_concat = caltech_ts_concat.dropna().reset_index(drop=True)

office001_ts_concat['Locations'] = 'Office 1'
jpl_ts_concat['Locations'] = 'JPL'
caltech_ts_concat['Locations'] = 'Caltech'

locations = pd.concat([office001_ts_concat, jpl_ts_concat, caltech_ts_concat], ignore_index=True)

In [ ]:
fig = plt.figure()
sns.histplot(x= 0,
             data=locations,
             bins=64,
             color ="navy",
             hue="Locations",
             alpha=1)
plt.ylabel("Frequency")
plt.xlabel("Charging power [kWh]")
plt.title("Histogram ACN datasets")
plt.rc('axes', axisbelow=True)
plt.grid(True)
plt.show()